In [1]:
import pandas as pd
import numpy as np
import math
from statistics import mean
import pickle
from scipy import sparse
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
import re
import geopy.distance

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


In [2]:
# double check the path on harddrive
import pathlib
print(pathlib.Path().resolve())

/Users/kennethshinn/Documents/fyoudie/python_workbooks


In [3]:
# this is critical and sensitive BING API stuff!!!

import csv
import requests
import json
import math

api_key = "AgUSM867a3r-7GFqCUQ81nvWLngFtrbanaBzA41qVEDoN-PFSEQiCTtx0eY9aJ--"

In [4]:
# location functions

def get_address(name, lattitude, longitude):
    lat_long_string = str(lattitude) + ',' + str(longitude)
    response = requests.get("https://dev.virtualearth.net/REST/v1/LocalSearch/",
                            params={"query": name,
                                    "userLocation": lat_long_string,
                                    "key": api_key})
    data = response.json()
    location_data = data['resourceSets'][0]['resources'][0]["Address"]
    return location_data


def get_current_location(lattitude, longitude):
    lat_long_string = str(lattitude) + ',' + str(longitude)
    response = requests.get("https://dev.virtualearth.net/REST/v1/Locations/" + lat_long_string + "?",
                            params={"key": api_key})
    data = response.json()
    location_data = data['resourceSets'][0]['resources'][0]["address"]
    return location_data


def get_distance_time_name(name, lattitude, longitude, trans_method):
    lat_long_string = str(lattitude) + ',' + str(longitude)
    dest_address = get_address(name, lattitude, longitude)["formattedAddress"]

    if (trans_method == "Driving") | (trans_method == "Transit"):
        optimization = "time"
    else:
        optimization = "distance"

    response = requests.get("https://dev.virtualearth.net/REST/v1/Routes/" + str(trans_method) + "?",
                            params={"o": "json",
                                    "wp.0": lat_long_string,
                                    "wp.1": dest_address,
                                    "distanceUnit": "Mile",
                                    "optimize": optimization,
                                    "key": api_key})
    data = response.json()
    direction_data = data['resourceSets'][0]["resources"][0]
    return direction_data

def get_distance_time_address(address, lattitude, longitude, trans_method):
    lat_long_string = str(lattitude) + ',' + str(longitude)

    if (trans_method == "Driving") | (trans_method == "Transit"):
        optimization = "time"
    else:
        optimization = "distance"

    response = requests.get("https://dev.virtualearth.net/REST/v1/Routes/" + str(trans_method) + "?",
                            params={"o": "json",
                                    "wp.0": lat_long_string,
                                    "wp.1": address,
                                    "distanceUnit": "Mile",
                                    "optimize": optimization,
                                    "key": api_key})
    data = response.json()
    direction_data = data['resourceSets'][0]["resources"][0]
    return direction_data

# returns nice values for distance (in miles) and time (in minutes of travel)
def get_distance_time_name_values(name, lattitude, longitude, trans_method):
    results = get_distance_time_name(name, lattitude, longitude, trans_method)
    distance = math.ceil(results["travelDistance"] * 10) / 10
    time = math.ceil(results["travelDuration"] / 60)
    return distance, time

# returns nice values for distance (in miles) and time (in minutes of travel)
def get_distance_time_address_values(address, lattitude, longitude, trans_method):
    results = get_distance_time_address(address, lattitude, longitude, trans_method)
    distance = math.ceil(results["travelDistance"] * 10) / 10
    time = math.ceil(results["travelDuration"] / 60)
    return distance, time



In [5]:
import json

# import restaurant data from files

business_file_path = "../yelp_dataset/yelp_academic_dataset_business.json"

business_info_dict = {}
with open(business_file_path, 'rb') as business_data:
    for b in business_data:
        b_dict = json.loads(b)

        bid = b_dict['business_id']
        category = b_dict["categories"]
        if (category is not None and "Restaurants" in category):
            business_info_dict[bid] = b_dict

business_info_df = pd.DataFrame(business_info_dict.values())

In [6]:
rid_dict = pd.read_pickle("../script_data/new_to_old_restaurant_ids.pkl")
rid_dict = {v:k for k,v in rid_dict.items()}

In [7]:
with open("../pickles/very_big_masks.pkl","rb") as f:
    mask_dict = pickle.load(f)

# # need to process this to be able to use it
# mask_vectors = list(mask_dict.values())
mask_vectors = [i.todense() for i in mask_dict]
mask_matrix = np.stack(mask_vectors)

/var/folders/kk/pr3cm19s0pnf27_x6wbg037m0000gn/T/ipykernel_14286/4280266897.py:2: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  mask_dict = pickle.load(f)


In [8]:
# this will pull the feature vectors for the restaurants

with open('../pickles/rid_feature_dict.pkl',"rb") as f:
    rid_feature_dict = pickle.load(f)
    
# this will pull the feature reweights (since due to the masks, some of the weights over power)
with open('../pickles/feature_weight_corrections.pkl',"rb") as f:
    feature_reweights = pickle.load(f)

In [9]:
feature_reweights

matrix([[0.33717928],
        [0.34531634],
        [0.34335799],
        [0.36251345],
        [0.5865434 ],
        [1.        ],
        [0.27171569],
        [0.41962033]])

In [10]:
# we have to post process the vectors so that we extract a parameter term that is weighted or unweighted
# price = weighted
# taste = weighted 
# noise = unweighted
# decor = weighted
# saltiness = unweighted
# msg = unweighted 
# cleaniless = weighted
# oiliness = unweighted

weighted = np.matrix([1,1,0,1,0,0,1,0])
unweighted = np.matrix([1,1,1,1,1,1,1,1]) - weighted

rid_feature_dict_combined = {}
for key, value in rid_feature_dict.items():
    # here, we apply the feature reweights
    rid_feature_dict_combined[key] = np.multiply(feature_reweights.T, np.multiply(unweighted, value[0].T) +  np.multiply(weighted, value[1].T))


In [11]:
# matching these with the people vectors, test
from numpy import genfromtxt
user_features = np.matrix(genfromtxt("../user_restaurant_data/user_features_test.csv", delimiter=','))
# normalize the user features if they aren't already
user_features = user_features / np.linalg.norm(user_features)


# Order is price, taste, noise, decor, saltiness, msg, cleanliness, and oiliness
# for every person in this file, dot the person with the feature vectors, and sort
user_sorted_rid = {}
cnt = 0
for row in user_features:
    print(row)
    user_feature_vector = row
    rid_scores = []
    for key, value in rid_feature_dict_combined.items():
        rid_scores.append((key, float(np.dot(user_feature_vector,value.T))))
    user_sorted_rid[cnt] = sorted(rid_scores, key=lambda x: x[1], reverse = True)
    cnt += 1

[[0.68599434 0.68599434 0.         0.17149859 0.         0.
  0.17149859 0.        ]]


In [12]:
# lattitude, longitude
user_locations = np.matrix(genfromtxt("../user_restaurant_data/user_locations_test.csv", delimiter=','))


In [13]:
person_no = 0

In [14]:
# convert the user sorted rids into a pandas data frame
rids = [x for (x,y) in user_sorted_rid[person_no]]
scores = [y for (x,y) in user_sorted_rid[person_no]]
    
data = {'rid': rids, 'score': scores}
rids_scores = pd.DataFrame.from_dict(data)

In [15]:
rids = []
business_ids = []

for key, value in rid_dict.items():
    rids.append(key)
    business_ids.append(value)
    
data = {'rid': rids, 'business_id': business_ids}
rids_ogrids = pd.DataFrame.from_dict(data)

In [16]:
# this is the master recommendations

df_master_recs_all = pd.merge(business_info_df, pd.merge(rids_scores, rids_ogrids, on='rid', how='inner'), on='business_id', how = 'inner')



In [17]:
restaurant_locations = list(zip(list(df_master_recs_all['latitude']),list(df_master_recs_all['longitude'])))

distances = []
for lat, long in restaurant_locations:
    coords_1 = (lat, long)
    coords_2 = (np.array(user_locations)[0][0], np.array(user_locations)[0][1])
    distances.append(geopy.distance.distance(coords_1, coords_2).mi)

df_master_recs_all['distance'] = distances

In [18]:
# get all restaurants within the distance threshold
distance_threshold = 20 # miles
df_master_recs = df_master_recs_all.loc[df_master_recs_all['distance'] < distance_threshold]

In [19]:
df_master_recs = df_master_recs.sort_values(by=['score'],ascending=False).head(10)

In [20]:
# add the driving distance to all possible recs
rec_names = list(df_master_recs['name'])

travel_distance = []
travel_times  = []

lat = np.array(user_locations)[0][0]
lng = np.array(user_locations)[0][1]
travel_method = 'Driving'

for name in rec_names:
    try: # first try querying just the name
        distance, time = get_distance_time_name_values(name, lat, lng, travel_method)
        travel_distance.append(distance)
        travel_times.append(time)
    except:
        try: # try to query the address
            address = df_master_recs.iloc[2]['address']
            city = df_master_recs.iloc[2]['city']
            state = df_master_recs.iloc[2]['state']
            address_string = address + ', ' + city + ', ' + state
            distance, time = get_distance_time_address_values(address_string, lat, lng, travel_method)
            travel_distance.append(distance)
            travel_times.append(time)
        except: # failure, just add None
            travel_distance.append(None)
            travel_times.append(None)
            

In [21]:
df_master_recs['travel_distance'] = travel_distance
df_master_recs['travel_time'] = travel_times

In [22]:
df_master_recs['score_rank'] = list(df_master_recs['score'].rank(ascending=False))
df_master_recs['travel_time_rank'] = list(df_master_recs['travel_time'].rank(ascending=True))
df_master_recs['combined_rank'] = df_master_recs['score_rank'] + df_master_recs['travel_time_rank']
df_master_recs = df_master_recs.sort_values(by=['combined_rank'],ascending=True)

In [23]:
df_master_recs

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,categories,hours,rid,score,distance,travel_distance,travel_time,score_rank,travel_time_rank,combined_rank
42379,AFjTPvLlpcoPWsTtK7Im2A,Kathi Rolls,4205 Curry Ford Rd,Orlando,FL,32806,28.524692,-81.332293,4.5,99,...,"Food, Halal, Wraps, Pizza, Indian, Juice Bars ...","{'Tuesday': '12:30-19:30', 'Wednesday': '12:30...",21838,0.046362,11.280752,0.8,3,4.0,1.0,5.0
40549,UVwHN6jxwIMGqjw-MfEuxA,La Granja,2809 W Vine St,Kissimmee,FL,34741,28.304838,-81.432153,4.0,41,...,"Desserts, Seafood, Latin American, Restaurants...",None,885,0.050586,12.556625,8.3,20,2.0,4.0,6.0
35492,ewGeqOJShmU9B3Wph0ckKQ,Taco Maker Mexican Grill,"9924 Universal Blvd, Ste 204",Orlando,FL,32819,28.427343,-81.444636,3.5,42,...,"Restaurants, Mexican, Food, Latin American, Br...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-20:0', ...",1178,0.046322,5.058130,6.8,18,5.0,2.0,7.0
24468,hwqH2wGy_FnCcRHtQ-quPw,Rana's Tikka House,1155 W State Rd 434,Longwood,FL,32750,28.698830,-81.365119,4.5,6,...,"Restaurants, Pakistani, Halal, Indian","{'Monday': '11:0-21:0', 'Tuesday': '11:0-21:0'...",5924,0.059235,17.763611,25.1,32,1.0,8.0,9.0
10077,xN0aHC1-tktqy54jS1OeVA,Tu Casa Restaurant,9251 S Orange Blossom Trl,Orlando,FL,32837,28.427921,-81.404156,4.5,293,...,"Mexican, Restaurants, Caribbean, Latin American","{'Monday': '0:0-0:0', 'Tuesday': '7:0-20:0', '...",40199,0.045902,7.113817,10.5,19,7.0,3.0,10.0
8862,KVReSFb2dwgjnSkl1Kn2vQ,kreyol kafe & Bakery,"12014 E Colonial Dr, Ste 120",Orlando,FL,32826,28.565845,-81.203960,4.5,54,...,"Caribbean, Food, Bakeries, Restaurants, Haitian","{'Monday': '11:0-18:0', 'Tuesday': '11:0-20:0'...",40657,0.047297,19.580073,24.8,33,3.0,9.5,12.5
50598,vEK16RsDLiMJrw9MXH5K-Q,China Bell Asian Restaurant,4382 W Vine St,Kissimmee,FL,34746,28.302817,-81.456469,4.0,57,...,"Restaurants, Chinese",None,2835,0.046170,12.219874,13.4,29,6.0,7.0,13.0
1148,b5eA9mluSKUngvkJ4ZCILA,Chimi King Buffet,5731 S Orange Blossom Trl,Orlando,FL,32839,28.475253,-81.396198,4.5,6,...,"Restaurants, Caribbean, Latin American, Buffet...","{'Monday': '10:0-19:0', 'Tuesday': '10:0-19:0'...",30655,0.044511,6.844814,9.8,24,9.0,5.0,14.0
47996,XDJj6UDRMl-rIormcjX8jA,JJ's Fresh From Scratch,2950 Curry Ford Rd,Orlando,FL,32806,28.523522,-81.345183,4.5,376,...,"Mediterranean, Venezuelan, Mexican, Lebanese, ...","{'Monday': '0:0-0:0', 'Tuesday': '11:0-18:0', ...",28137,0.044258,10.511363,15.6,25,10.0,6.0,16.0
12403,LL6l5YHvRZ_UJaFo1JxmJQ,Mi Parrillita,8557 W Irlo Bronson Memorial Hwy,Kissimmee,FL,34747,28.347665,-81.631501,5.0,31,...,"Fast Food, Peruvian, Food, Burgers, Food Truck...","{'Monday': '0:0-0:0', 'Tuesday': '18:0-1:0', '...",1759,0.045873,11.480246,24.8,33,8.0,9.5,17.5
